## CIFAR10 Hparam search

In [1]:
import torch
import numpy as np
import os, sys
import shutil
import pickle
import cv2
from tqdm import tqdm
from sklearn.cluster import KMeans
from datetime import datetime

sys.path.insert(0, 'src')
from utils.utils import ensure_dir, read_json, informal_log
from utils.visualizations import plot
from utils.model_utils import prepare_device

import model.metric as module_metric
import model.loss as module_loss
import datasets.datasets as module_data
import model.model as module_model

from train import main as train_fn
from predict import predict
from parse_config import ConfigParser


sys.path.insert(0, 'setup')
from setup_cifar10 import setup_cifar10 
# import cv2
# print(cv2.__version__)

In [2]:
config_path = 'configs/train_cifar10_sift_explainer.json'
debug = False
if debug:
    learning_rates = [1e-6] #, 1e-5, 1e-4, 1e-3, 5e-2, 1e-2, 5e-1, 1e-1]
    weight_decays = [0, 1e-1] #, 1e-2, 1e-3]
else:
    learning_rates = [1e-4, 1e-3, 5e-2, 1e-2, 5e-1, 1e-1]
    weight_decays = [0, 1e-1, 1e-2, 1e-3]

config_json = read_json(config_path)


### Set up data loaders

In [3]:
dataset_args = config_json['dataset']['args']
train_descriptors_dataset = module_data.KDDataset(split='train', **dataset_args)
test_descriptors_dataset = module_data.KDDataset(split='test', **dataset_args)

dataloader_args = config_json['data_loader']['args']
train_descriptors_dataloader = torch.utils.data.DataLoader(
    train_descriptors_dataset,
    shuffle=True,
    **dataloader_args)
test_descriptors_dataloader = torch.utils.data.DataLoader(
    test_descriptors_dataset,
    shuffle=False,
    **dataloader_args)

device, device_ids = prepare_device(config_json['n_gpu'])
metric_fns = [getattr(module_metric, met) for met in config_json['metrics']]
loss_fn = getattr(module_loss, config_json['loss'])

### Run Hyperparameter search

In [ ]:
best = {
    'lr': -1,
    'wd': -1,
    'val_acc': -1
}
n_trials = len(learning_rates) * len(weight_decays)
trial_idx = 1
timestamp = datetime.now().strftime(r'%m%d_%H%M%S')

# Logging
log_path = os.path.join(config_json['trainer']['save_dir'], timestamp, 'log.txt')
ensure_dir(os.path.dirname(log_path))
informal_log("Hyperparameter search", log_path)
informal_log("Learning rates: {}".format(learning_rates), log_path)
informal_log("Weight decays: {}".format(weight_decays), log_path)

# 

# Debug mode
if debug:
    config_json['trainer']['epochs'] = 1
    
for lr in learning_rates:
    for wd in weight_decays:
        # Update config json
        config_json['optimizer']['args'].update({
            'lr': lr,
            'weight_decay': wd
        })
        
        # Create run ID for trial
        itr_timestamp = datetime.now().strftime(r'%m%d_%H%M%S')
        informal_log("[{}] Trial {}/{}: LR = {} WD = {}".format(
            itr_timestamp, trial_idx, n_trials, lr, wd), log_path)
        run_id = os.path.join(timestamp, 'trials', 'lr_{}-wd_{}'.format(lr, wd))
        config = ConfigParser(config_json, run_id=run_id)
        print(config.config['optimizer']['args'])
        
        # Train model
        model = train_fn(
            config=config, 
            train_data_loader=train_descriptors_dataloader,
            val_data_loader=test_descriptors_dataloader)
        
        # Restore model
        model_restore_path = os.path.join(config.save_dir, 'model_best.pth')
        
        model.restore_model(model_restore_path)
        print("restored model")
        # Run on validation set using predict function
        
        trial_path = os.path.dirname(os.path.dirname(model_restore_path))
        output_save_path = os.path.join(trial_path, "val_outputs.pth")
        log_save_path = os.path.join(trial_path, "val_metrics.pth")
        
        validation_data = predict(
            data_loader=test_descriptors_dataloader,
            model=model,
            metric_fns=metric_fns,
            device=device,
            loss_fn=loss_fn,
            output_save_path=output_save_path,
            log_save_path=log_save_path)
       
        # Obtain accuracy and compare to previous best
        print(validation_data['metrics'].keys())
        val_accuracy = validation_data['metrics']['accuracy']
        if val_accuracy > best['val_acc']:
            best.update({
                'lr': lr,
                'wd': wd,
                'val_acc': val_accuracy
            })
            informal_log("Best accuracy of {:.3f} with lr={} and wd={}".format(val_accuracy, lr, wd), log_path)
            informal_log("Trial path: {}".format(trial_path), log_path)
            # Copy model and outputs to 1 directory for easy access
            best_save_dir = os.path.join(os.path.dirname(os.path.dirname(trial_path)), 'best')
            ensure_dir(best_save_dir)
            best_outputs_save_path = os.path.join(best_save_dir, 'outputs.pth')
            best_model_save_path = os.path.join(best_save_dir, 'model.pth')
            torch.save(validation_data['logits'], best_outputs_save_path)
            model.save_model(best_model_save_path)
            informal_log("Saved model and outputs to {}".format(best_save_dir), log_path)
            
            
        trial_idx += 1

Hyperparameter search
Learning rates: [0.0001, 0.001, 0.05, 0.01, 0.5, 0.1]
Weight decays: [0, 0.1, 0.01, 0.001]
[0526_132630] Trial 1/24: LR = 0.0001 WD = 0
OrderedDict([('lr', 0.0001), ('weight_decay', 0), ('amsgrad', False)])
Created LinearLayers model with 510 trainable parameters
Training from scratch.
Checkpoint save directory: saved/cifar10/resnet18/explainer/sift_descriptor_histogram/cifar128_128/hparam_search/0526_132630/trials/lr_0.0001-wd_0/models
Train Epoch: 1 [0/196 (0%)] Loss: 2.312195
Train Epoch: 1 [48/196 (24%)] Loss: 2.303904
Train Epoch: 1 [96/196 (49%)] Loss: 2.299834
Train Epoch: 1 [144/196 (73%)] Loss: 2.295412
Train Epoch: 1 [192/196 (98%)] Loss: 2.310294
    epoch          : 1
    val_TP         : [  0 297  21  25 657   0   2  29   0   8]
    val_TN         : [9000 6563 8935 8875 3195 9000 8943 8620 9000 8908]
    val_FPs        : [   0 2437   65  125 5805    0   57  380    0   92]
    val_FNs        : [1000  703  979  975  343 1000  998  971 1000  992]
    val

/n/fs/ac-alignment/explain-alignment/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-alignment/explain-alignment/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Train Epoch: 2 [0/196 (0%)] Loss: 2.310613
Train Epoch: 2 [48/196 (24%)] Loss: 2.306568
Train Epoch: 2 [96/196 (49%)] Loss: 2.300458
Train Epoch: 2 [144/196 (73%)] Loss: 2.300722
Train Epoch: 2 [192/196 (98%)] Loss: 2.292264
    epoch          : 2
    val_TP         : [  0 358  44  59 626   0   2  43   0  26]
    val_TN         : [9000 6540 8861 8720 3813 9000 8902 8526 9000 8796]
    val_FPs        : [   0 2460  139  280 5187    0   98  474    0  204]
    val_FNs        : [1000  642  956  941  374 1000  998  957 1000  974]
    val_accuracy   : 0.1158
    val_per_class_accuracy: [0.9    0.6898 0.8905 0.8779 0.4439 0.9    0.8904 0.8569 0.9    0.8822]
    val_per_class_accuracy_mean: 0.82316
    val_precision  : [0.         0.12704045 0.24043716 0.1740413  0.10768966 0.
 0.02       0.08317215 0.         0.11304348]
    val_precision_mean: 0.08654241969951267
    val_recall     : [0.    0.358 0.044 0.059 0.626 0.    0.002 0.043 0.    0.026]
    val_recall_mean: 0.11580000000000001
    val

/n/fs/ac-alignment/explain-alignment/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-alignment/explain-alignment/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Train Epoch: 3 [0/196 (0%)] Loss: 2.302526
Train Epoch: 3 [48/196 (24%)] Loss: 2.301350
Train Epoch: 3 [96/196 (49%)] Loss: 2.305632
Train Epoch: 3 [144/196 (73%)] Loss: 2.295575
Train Epoch: 3 [192/196 (98%)] Loss: 2.302958
    epoch          : 3
    val_TP         : [  0 405  74  91 591   0   3  47   0  52]
    val_TN         : [9000 6452 8747 8567 4496 9000 8857 8475 9000 8669]
    val_FPs        : [   0 2548  253  433 4504    0  143  525    0  331]
    val_FNs        : [1000  595  926  909  409 1000  997  953 1000  948]
    val_accuracy   : 0.1263
    val_per_class_accuracy: [0.9    0.6857 0.8821 0.8658 0.5087 0.9    0.886  0.8522 0.9    0.8721]
    val_per_class_accuracy_mean: 0.8252600000000001
    val_precision  : [0.         0.13714866 0.22629969 0.17366412 0.11599607 0.
 0.02054795 0.08216783 0.         0.13577023]
    val_precision_mean: 0.08915945656474318
    val_recall     : [0.    0.405 0.074 0.091 0.591 0.    0.003 0.047 0.    0.052]
    val_recall_mean: 0.12630000000000

...

Train Epoch: 7 [0/196 (0%)] Loss: 2.294180
Train Epoch: 7 [48/196 (24%)] Loss: 2.293647
Train Epoch: 7 [96/196 (49%)] Loss: 2.293853
Train Epoch: 7 [144/196 (73%)] Loss: 2.294265
Train Epoch: 7 [192/196 (98%)] Loss: 2.288751
    epoch          : 7
    val_TP         : [  0 417 208 229 410   0  52 106   5 217]
    val_TN         : [9000 7059 8202 7894 6521 9000 8572 8380 8993 8023]
    val_FPs        : [   0 1941  798 1106 2479    0  428  620    7  977]
    val_FNs        : [1000  583  792  771  590 1000  948  894  995  783]
    val_accuracy   : 0.1644
    val_per_class_accuracy: [0.9    0.7476 0.841  0.8123 0.6931 0.9    0.8624 0.8486 0.8998 0.824 ]
    val_per_class_accuracy_mean: 0.8328800000000001
    val_precision  : [0.         0.17684478 0.20675944 0.17153558 0.14191762 0.
 0.10833333 0.14600551 0.41666667 0.18174204]
    val_precision_mean: 0.1549804979325412
    val_recall     : [0.    0.417 0.208 0.229 0.41  0.    0.052 0.106 0.005 0.217]
    val_recall_mean: 0.1644
    val_pr

/n/fs/ac-alignment/explain-alignment/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-alignment/explain-alignment/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Train Epoch: 8 [0/196 (0%)] Loss: 2.291494
Train Epoch: 8 [48/196 (24%)] Loss: 2.296324
Train Epoch: 8 [96/196 (49%)] Loss: 2.286265
Train Epoch: 8 [144/196 (73%)] Loss: 2.286866
Train Epoch: 8 [192/196 (98%)] Loss: 2.288881
    epoch          : 8
    val_TP         : [  0 396 231 248 379   0  74 114  21 258]
    val_TN         : [8999 7251 8086 7807 6863 8998 8477 8367 8980 7893]
    val_FPs        : [   1 1749  914 1193 2137    2  523  633   20 1107]
    val_FNs        : [1000  604  769  752  621 1000  926  886  979  742]
    val_accuracy   : 0.1721
    val_per_class_accuracy: [0.8999 0.7647 0.8317 0.8055 0.7242 0.8998 0.8551 0.8481 0.9001 0.8151]
    val_per_class_accuracy_mean: 0.8344199999999999
    val_precision  : [0.         0.18461538 0.20174672 0.17210271 0.15063593 0.
 0.1239531  0.15261044 0.51219512 0.18901099]
    val_precision_mean: 0.1686870397564508
    val_recall     : [0.    0.396 0.231 0.248 0.379 0.    0.074 0.114 0.021 0.258]
    val_recall_mean: 0.172099999999999

/n/fs/ac-alignment/explain-alignment/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Train Epoch: 9 [0/196 (0%)] Loss: 2.289296
Train Epoch: 9 [48/196 (24%)] Loss: 2.285641
Train Epoch: 9 [96/196 (49%)] Loss: 2.288734
Train Epoch: 9 [144/196 (73%)] Loss: 2.288110
Train Epoch: 9 [192/196 (98%)] Loss: 2.282132
    epoch          : 9
    val_TP         : [  1 388 253 262 327   1 102 117  37 290]
    val_TN         : [8994 7321 7982 7741 7207 8990 8387 8404 8948 7804]
    val_FPs        : [   6 1679 1018 1259 1793   10  613  596   52 1196]
    val_FNs        : [999 612 747 738 673 999 898 883 963 710]
    val_accuracy   : 0.1778
    val_per_class_accuracy: [0.8995 0.7709 0.8235 0.8003 0.7534 0.8991 0.8489 0.8521 0.8985 0.8094]
    val_per_class_accuracy_mean: 0.8355600000000001
    val_precision  : [0.14285714 0.18771166 0.19905586 0.1722551  0.15424528 0.09090909
 0.14265734 0.16409537 0.41573034 0.19515478]
    val_precision_mean: 0.18646719623855698
    val_recall     : [0.001 0.388 0.253 0.262 0.327 0.001 0.102 0.117 0.037 0.29 ]
    val_recall_mean: 0.1778
    val_pre

### Check best model's results on training data

In [8]:
# Trained on soft labels with 75 clusters
# model_restore_path = 'saved/cifar10/resnet18/explainer/sift_descriptor_histogram/hparam_search/0525_103130/best/model.pth'
# Trained on hard GT labels with 75 clusters
model_restore_path = 'saved/cifar10/resnet18/explainer/sift_descriptor_histogram/true_labels/75_clusters/models/model_best.pth'
# Variable
model_restore_path = 'saved/cifar10/resnet18/explainer/sift_descriptor_histogram/mlp_512/true_labels/sift32_32_75means/models/model_best.pth'
model_args = config_json['arch']['args']
model = module_model.LinearLayers(
    checkpoint_path=model_restore_path,
    **model_args)

model.eval()
model = model.to(device)

# Evaluate on validation set as a sanity check
validation_results = predict(
        data_loader=test_descriptors_dataloader,
        model=model,
        metric_fns=metric_fns,
        device=device,
        loss_fn=loss_fn,
        output_save_path=None,
        log_save_path=None)

print("Validation accuracy: {}".format(validation_results['metrics']['accuracy']))

training_results = predict(
        data_loader=train_descriptors_dataloader,
        model=model,
        metric_fns=metric_fns,
        device=device,
        loss_fn=loss_fn,
        output_save_path=None,
        log_save_path=None)

print("Training accuracy: {}".format(training_results['metrics']['accuracy']))

100%|███████████████████████████████████████████| 40/40 [00:00<00:00, 80.11it/s]


Validation accuracy: 0.2436


100%|████████████████████████████████████████| 196/196 [00:00<00:00, 296.49it/s]


Training accuracy: 0.2519
